# Comparison of 3T and 7T data in epilepsy
-1. Concat zBrains outputs into dataframes
0. Demographics (Table 1)
1. zBrain/wBrain (surface)
    a. Histograms of vertex wise scores
        i. sub-comparisons with different smoothing kernels
    b. Quantifying extreme vertex groups
        i. number of identified abnormal areas
        ii. size of each abnormal area (number of adjacent extreme vertices)
2. Brainstats (surface)
    a. t-scores for 3T and 7T
    b. cohen's D map between 3T and 7T images

# -1. Prepare data

In [6]:
import sys
import importlib

import vrtx

sys.path.append('/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code/')  # Replace with the path to Utils
from Utils import gen


In [7]:
importlib.reload(vrtx)

<module 'vrtx' from '/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/Epilepsy_7T/zBrainsAnalyses/code/3T7T/analyses/vrtx.py'>

In [9]:
# IDs of interest
dir_IDs = "" # should be in long format

# zb output directories
# N.b. .surf.gii files usually in:
# root/zBrains_output/sub-ID/ses-xx/maps/region
root_dir = "/data/mica3/BIDS_PNI/derivatives" 
dir_zb = "zbrains_3T7T_daniel_8Jan2025" # that holds sub/ses
dir_sub = "maps"

cortex = {
    "region": "cortex",
    "surfaces": ["pial", "midthickness", "white"],
    "resolution": "32k",
    "features": ["ADC", "T1map", "thickness"], # (list) features to extract
    "smoothing": [2,5,10]
}

hippocampus = {
    "region": "hippocampus",
    "surfaces": ["midthickness"],
    "resolution": "0p5mm",
    "features": ["ADC", "T1map", "thickness"], # (list) features to extract
    "smoothing": [1,2,5]
}

subcortex = {
    "region": "subcortex",
    "features": ["ADC", "T1map", "thickness"],
    "smoothing": [2,5,10]
}

regions = [cortex, hippocampus, subcortex]



In [15]:
# build df (one per unique combination of feature, hemisphere, surface, smoothing)

file_ptrns = []  # Initialize as an empty list

for region in regions:
    region_ptrns = []  # Initialize as an empty list
    
    region_ptrns = vrtx.zbFilePtrn(region)

    # save patterns for each region in seperate list
    file_ptrns.append(region_ptrns)  # Append directly to the list

for region, i in zip(regions, range(len(file_ptrns))): print(f"{region["region"]}: {len(file_ptrns[i])}")
print('\n'.join(' '.join(map(str,sl)) for sl in file_ptrns))
print() # Print the list of file patterns


cortex: 54
hippocampus: 18
subcortex: 3
hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-2mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-T1map_smooth-2mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-thickness_smooth-2mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-5mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-T1map_smooth-5mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-thickness_smooth-5mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-T1map_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-thickness_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-2mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-2mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-thickness_smooth-2mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-5mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-

In [ ]:
# iterate through each file pattern and IDs_ses, get paths and extract paths to a df
IDs = Utils.gen.read(dir_IDs)
for i in range(IDs):
    ID = IDs[i,"ID"]
    SES = IDs[i,"SES"]
    for region, region_ptrns in zip(regions, file_ptrns):
        for ptrn in region_ptrns:
            dir = "/".join([root_dir, dir_zb, ID, SES, dir_sub, region, ptrn])
            # read in this path, save into a column in df

        # save df to csv
        

## 1. zBrain
### a. Histograms
